In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_excel('data_1984-2000.xlsx')
df = df[df['摘要(译)(English)'] != '-']
df1 = df.dropna(subset=['摘要(译)(English)'])
df1

,公开(公告)号,标题,法律状态/事件,摘要(译)(简体中文),摘要(译)(English),标题(译)(简体中文),标题(译)(English),[标]原始申请(专利权)人,原始申请(专利权)人国家,申请年,...,被引用专利,引用专利,引用专利数量,被引用专利数量,专利类型,摘要,权利要求,申请号,原始申请(专利权)人地址,国家
0,SA120420254A,Frame for an interactive picture of agriculture,授权,"本发明涉及一种用于农业交互式图像的框架1,其将审美功能与在其内部种植的可能性相结合,因为它由...",The present invention relates to a frame for a...,农业互动图片框架,Frame for an interactive picture of agriculture,SHADI KHAMIS SALIM ALAMAYREH,JO,-,...,-,-,0,0,发明申请,The present invention relates to a frame for a...,-,SA120420254,NaN,Jordan
1,KR1019810000834B1,3-페닐-5-치환-4(1H)-피리돈(티온)의 제조방법,权利终止,无内容。,"Title compds. (I; R = Me, Et, acetoxy, OH; R1 ...",3-苯基-5-取代-4(1h)-吡啶酮(硫酮)的制备方法,Process for preparing 3-phenyl-5-substituted-4...,美国礼来大药厂,US,1978,...,-,-,0,0,授权发明,내용 없음._x000D_\n,의 공란은 화합물을 명명된 종에 대해 시험하지 않은 것이다. 어떤 경우에는 식물종에...,KR1019780001933,NaN,United States
2,DE2935036A1,Pflanzverfahren und Pflanzgut hierfür,驳回,"该系统用于在花园、公园或屋顶种植大面积植物,使用生长在地面上的植物的短插条 (1),每个植物...",The system is intended for planting large area...,"大面积种植系统 - 将带有枝条的插条撒在堆肥上,并用沙子和/或堆肥轻轻覆盖它们",Planting system for large areas - scatters cut...,ZINK WALTER,DE,1979,...,DE3637463A1 | DE3637463C2 | DE3708491A1,AT231212B | DE2265298A | DE2265298B2 | DE25006...,6,3,发明申请,The system is intended for planting large area...,[0001] PatenLansprüche Pflanzenverfahren für ...,DE2935036,NaN,Germany
4,DE3026919A1,"Nutzung von Sonnenenergie, die in aufgelöster ...",撤回-未指定类型,太阳能可以通过在围绕太阳的轨道上并相对靠近太阳的反射器聚焦其分散的光线来使用。 这些将与地球...,The sun energy can be used by focussing its di...,"太阳能收集系统——使用靠近太阳的反射器,同步。 通过地球运动将能量分配到废物和敌对区域",Solar energy collection system - uses reflecto...,WEBER ERNST ING GRAD,DE,1980,...,-,-,0,0,发明申请,The sun energy can be used by focussing its di...,"[0001] Nutzung von Sonnenenergie, die in aufge...",DE3026919,NaN,Germany
5,DE3102523C2,Gegenstromwärmetauscher,期限届满,"本发明涉及一种根据逆流原理工作的热交换器,其通流通道由相互连接的单个蜂窝状元件组成。 这些元...",The invention relates to heat exchangers which...,逆流换热器,counterflow heat exchanger,PENZKOFER LUDWIG | SCHONHAMMER JOHANN,DE,1981,...,DE102006036965A1 | DE19647353B4 | DE19737158A1...,DE2523151A1,1,4,授权发明,"Gegenstand der Erfindung sind Wärmetauscher, d...",Patentansprüche:_x000D_\n1. Gegenstromwärmetau...,DE3102523,NaN,Germany
6,EP0084289A3,"Climate controlled floor for agricultural, ind...",撤回-视为撤回,Der Klimaboden ist eine Erfindung im Gebiet de...,Der Klimaboden ist eine Erfindung im Gebiet de...,"用于农业、工业和住宅目的的气候控制地板,包括加热板或带有灌溉系统的空心板,及其安装系统","Climate controlled floor for agricultural, ind...",ENGEL SYLVIE,FR,1982,...,FR2580145A1,CA1061774A | CH497111A | CH603029A5 | DE250179...,7,1,发明申请,Der Klimaboden ist eine Erfindung im Gebiet de...,1. Klimaboden für die Beheizung von Pflanzen b...,EP1982810268,NaN,France
7,DE3319598A1,Isolierbienenkasten mit Hinterlüftung und Rege...,撤回-视为撤回,带后部通风的隔热蜂箱具有耐候外壁、隔热层、用于控制热量和湿度的气井和具有吸湿性能的呼吸壁。 ...,The insulating beehive with rear ventilation h...,带有后部通风和控制的绝缘蜂箱,Insulating beehive with rear ventilation and c...,BUSCHACK WILHELM,DE,1983,...,AU713985B2 | EP3175703A3 | EP3366123A1 | EP336...,-,0,12,发明申请,The insulating beehive with rear ventilation h...,[0001] P a t e n t a n s p r ü c h e 1. Isolie...,DE3319598,NaN,Germany
8,YU19831022A,Prefabricated hall with a system of rectilinea...,期限届满,"一个装配有连续飞檐和拱形支撑系统的礼堂,采用型材和连接方法,实现特别简单的构造以及从建筑物接...",An assembly hall with a system of continuous c...,"预制大厅,带有相互连接的直线和弯曲异形支撑梁系统,以形成特别轻的结构并从中接收和输送冷凝水",Prefabricated hall with a system of rectilinea...,SURKOVIC DORAN,RS,1983,...,-,-,0,0,发明申请,Montažna hala sa sistemom kontinualnih rožnjač...,该数据不支持导出,YU19831022,NaN,Serbia
9,DE3344945A1,Verfahren und Vorrichtung zur erdlosen Aufzuch...,撤回-视为撤回,"本发明涉及一种用于无土栽培植物的方法和设备,最好是在露天,在通过固定和密封地表而形成的倾斜平...",The invention relates to a method and device f...,"在倾斜平面上,优选在露天进行无土栽培和栽培植物的方法和装置",Method and device for soilless raising and cul...,GRAF VON WOLKENSTEIN PATENTVERWGMBH I G,DE,1983,...,CN103917086A | CN108668875A | DE3633390A1 | GB...,DE2641945A1 | DE2701870A1 | DE2724559A1 | DE28...,10,5,发明申请,The invention relates to a method and device f...,

In [2]:
def truncate_text(text, max_length=510, redundancy=20):
    if len(text) <= max_length:
        return text
    
    end_idx_cn = text.rfind('。', 0, max_length)
    end_idx_en = text.rfind('.', 0, max_length)
    
    end_idx = max(end_idx_cn, end_idx_en)
    
    if end_idx == -1:
        start_idx = max_length - redundancy if max_length > redundancy else 0
        end_idx = max_length + redundancy if len(text) > max_length + redundancy else len(text)
        return text[start_idx:end_idx]
    else:
        return text[:end_idx + 1]

df1['摘要(译)(English)'] = df1['摘要(译)(English)'].apply(lambda x: truncate_text(x, max_length=512))

In [3]:
df1 = df1[['摘要(译)(English)']]

In [4]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

device = torch.device("cpu")

tokenizer = DistilBertTokenizer.from_pretrained(r'distilbert-base-nli-mean-tokens')

model = DistilBertModel.from_pretrained(r'distilbert-base-nli-mean-tokens')
model = model.to(device)

def get_text_embedding(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :]
    return embedding

text = df1['摘要(译)(English)'].values.tolist()
embeddings = get_text_embedding(text)
embeddings = embeddings.detach().cpu().numpy()
print(embeddings.shape)

(50, 768)


In [5]:
import umap
umap_model = umap.UMAP(n_components=3, random_state=2022)

umap_embeddings = umap_model.fit_transform(embeddings)
umap_embeddings.shape

(50, 3)

In [6]:
from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=1, random_state=2024).fit(umap_embeddings)
labels = cluster.labels_

df['cluster'] = labels
df1['cluster'] = cluster.labels_
df1.head()
df2 = df1
df2

,摘要(译)(English),cluster
0,The present invention relates to a frame for a...,0
1,"Title compds. (I; R = Me, Et, acetoxy, OH; R1 ...",0
2,The system is intended for planting large area...,0
4,The sun energy can be used by focussing its di...,0
5,The invention relates to heat exchangers which...,0
6,Der Klimaboden ist eine Erfindung im Gebiet de...,0
7,The insulating beehive with rear ventilation h...,0
8,An assembly hall with a system of continuous c...,0
9,The invention relates to a method and device f...,0
10,A working method for the harvesting of field c...,0


In [7]:
import re
from nltk.corpus import stopwords
def filter_text(text):    
    text = re.sub(r'\([^)]*\)', '', text)
    pattern = re.compile(r'[^a-zA-Z\s]')
    text = pattern.sub(' ', text)
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(filtered_words)
df2['摘要(译)(English)'] = df2['摘要(译)(English)'].astype(str)
df2['摘要(译)(English)'] = [filter_text(item) for item in df2['摘要(译)(English)']]

In [8]:
df3 = df2
df3

,摘要(译)(English),cluster
0,The present invention relates frame interactiv...,0
1,Title compds useful herbicide prepd reacting c...,0
2,The system intended planting large areas garde...,0
4,The sun energy used focussing dispersed rays r...,0
5,The invention relates heat exchangers operate ...,0
6,Der Klimaboden ist eine Erfindung im Gebiet de...,0
7,The insulating beehive rear ventilation weathe...,0
8,An assembly hall system continuous cornices ar...,0
9,The invention relates method device soilless r...,0
10,A working method harvesting field crops optimu...,0


In [9]:
df3['cluster'] = cluster.labels_
df3.head()

,摘要(译)(English),cluster
0,The present invention relates frame interactiv...,0
1,Title compds useful herbicide prepd reacting c...,0
2,The system intended planting large areas garde...,0
4,The sun energy used focussing dispersed rays r...,0
5,The invention relates heat exchangers operate ...,0


In [10]:
cluster0 = df3[df3['cluster']==0]
cluster0.head()

,摘要(译)(English),cluster
0,The present invention relates frame interactiv...,0
1,Title compds useful herbicide prepd reacting c...,0
2,The system intended planting large areas garde...,0
4,The sun energy used focussing dispersed rays r...,0
5,The invention relates heat exchangers operate ...,0


In [11]:
from keybert import KeyBERT

def get_top_n_words(text, top_n=10):
    model = KeyBERT(r'all-mpnet-base-v2')
    keywords = model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=top_n)
    return keywords

merged_text = ' '.join(cluster0['摘要(译)(English)'].tolist())

keywords = get_top_n_words(merged_text, top_n=10)
keywords

[('greenhouse', 0.5737),
 ('greenhouses', 0.5727),
 ('horticulture', 0.4688),
 ('irrigation', 0.4361),
 ('agriculture', 0.4325),
 ('cultivating', 0.4213),
 ('planting', 0.4155),
 ('agricultural', 0.4059),
 ('crops', 0.4058),
 ('phytoremediation', 0.3903)]

In [12]:
cluster0 = df3[df3['cluster']==0]
c0 = ' '.join(cluster0['摘要(译)(English)'].tolist())

In [13]:
df4 = []

common_words = get_top_n_words(c0, 10)
df4.append(common_words)

df5 = pd.DataFrame(df4)
df5 = df5.T
df5.columns = ['cluster0']
df5

,cluster0
0,"(greenhouse, 0.5737)"
1,"(greenhouses, 0.5727)"
2,"(horticulture, 0.4688)"
3,"(irrigation, 0.4361)"
4,"(agriculture, 0.4325)"
5,"(cultivating, 0.4213)"
6,"(planting, 0.4155)"
7,"(agricultural, 0.4059)"
8,"(crops, 0.4058)"
9,"(phytoremediation, 0.3903)"


In [14]:
df5.to_excel('result_1984-2000.xlsx',index=False)